In [ ]:
import re, os

import numpy as np
import cv2
from PIL import Image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import pickle

import pandas as pd

from stereo_func import StereoFunc, stereo_match

In [ ]:
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('default')

out_dir = 'bigStereoCluster3/'
results_set = {}
for fl in sorted(os.listdir(out_dir)):
    if 'pkl' not in fl:
        continue
    #if '_150_' in fl:
    #    continue
    with open(os.path.join(out_dir,fl),'rb') as fp:
        import_data = pickle.load(fp)
        if type(import_data) == list:
            results = import_data
        else:
            results = import_data['results']
    dim = int('_'.join(fl.split('_')[-2:-1]))
    algo = '_'.join(fl.split('_')[0:2])
    algo = algo.replace('single_cma','CMA')

    algo = algo.replace('posthoc_cma','post hoc')
    algo = algo.replace('staged_cma','staged')
    algo = algo.replace('onlinewin_cma','online')

    results_set[(algo,dim,fl)] = results

In [ ]:
[_ for _ in results_set.keys() if '_1.0_1.0_' not in _[2]],[_ for _ in results_set.keys() if '_150_' in _[2]]

In [ ]:
plt.figure(dpi=250,figsize=(6, 3))
dfs = [] 
for name in sorted(results_set):

    res = results_set[name]
    df = pd.DataFrame(res,columns=['res','base','oracle'])
    df['name'] = name[-1]
    df['method'] = name[0]
    df['iter'] = name[1]
    df['res'] = np.array(df[['res','base']]).min(axis=1)
    dfs.append(df)
    #mean_v = df.groupby('dim').mean()
    #min_v = df.groupby('dim').min()
df = pd.concat(dfs)
base_min_v = df.min(axis=0)

mean_df = df.groupby(['method','iter']).mean().reset_index()
min_df = df.groupby(['method','iter']).min().reset_index()

std_df = df.groupby(['method','iter']).std().reset_index()
cnt_df= df.groupby(['method','iter']).count().reset_index()

import seaborn as sns
from collections import defaultdict
with sns.color_palette('plasma', n_colors=len(df.method.unique())):
    span_v = base_min_v.base-base_min_v.oracle

    convert_f = lambda x: (base_min_v.oracle-x)/span_v+1
    for method in df.method.unique():
        mean_p = convert_f(mean_df[mean_df.method == method].res)
        diff = 1.96*std_df[std_df.method == method].res/np.sqrt(cnt_df[cnt_df.method == method].res)

        ub = convert_f(mean_df[mean_df.method == method].res-diff)
        lb = convert_f(mean_df[mean_df.method == method].res+diff)

        label = name[0]#.upper()
        ls_lookup = defaultdict(lambda: 'dashdot')
        ls_lookup['cma'] = '-'
        ls_lookup['post hoc'] = '--'
        ls_lookup['online'] = 'dotted'

        ls = ls_lookup[method.lower()]
        ms_lookup = defaultdict(lambda: '^')
        ms_lookup['CMA'] = '*'
        ms_lookup['online'] = 'v'
        ms_lookup['post hoc'] = 'x'
        ms = ms_lookup[method.lower()]
        label_p = method[0].upper() + method[1:]

        plt.plot(mean_df[mean_df.method == method].iter,mean_p,label=label_p,ls=ls,marker=ms,lw=2)
        plt.fill_between(mean_df[mean_df.method == method].iter,lb,ub,alpha=0.15)
    #plt.plot(mean_df[mean_df.method == method].iter,mean_df[mean_df.method == method].base,label="Initialization",c='k',ls='--')
    #plt.plot(mean_df[mean_df.method == method].iter,mean_df[mean_df.method == method].oracle,label="Oracle",c='r',ls='--')

plt.xlabel('Function Evaluations')
plt.ylabel('Quality of minima \n (0 is initialization, 1 is oracle)')
plt.title('Optimizing Stereo Matching Parameters')
plt.legend()
plt.tight_layout()
plt.legend(ncol=4,loc='lower right',fancybox=True,shadow=False,framealpha=0.5)
#plt.ylim(bottom=0)
plt.xlim(right=250)
plt.tight_layout()
plt.savefig('lol_stereo_newest.pdf')

In [ ]:
df[(df.iter == 150) & (df.method =='online')]

In [ ]:
base_min_v

In [ ]:
mean_df

In [ ]:
mean_df# 1.074519

In [ ]:
min_df

In [ ]:
df.groupby(['method','iter']).mean()

In [ ]:
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('default')

results_Algo = {}
for fl in sorted(os.listdir(out_dir)):
    if 'pkl' not in fl:
        continue
    #if '_150_' in fl:
    #    continue
    with open(os.path.join(out_dir,fl),'rb') as fp:
        import_data = pickle.load(fp)
        if type(import_data) == list:
            results = import_data
        else:
            results = import_data['results']
    dim = int('_'.join(fl.split('_')[-2:-1]))
    algo = '_'.join(fl.split('_')[0:2])
    algo = algo.replace('single_cma','CMA')

    algo = algo.replace('posthoc_cma','post hoc')
    algo = algo.replace('staged_cma','staged')
    algo = algo.replace('onlinewin_cma','online')

    results_Algo[(algo,dim,fl)] = import_data['clusters']

In [ ]:
import torch
from skimage.transform import resize
model = torch.hub.load('pytorch/vision', 'squeezenet1_1', pretrained=True)
model.classifier = torch.nn.AdaptiveAvgPool2d((1,1))
_ = model.eval()


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
def fix_img(array):
    vec = resize(array,(224,224)).astype(np.float32)
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    vec = (vec-mean)/std
    return vec
cfunc = StereoFunc()
conv_imgs = [fix_img(d_img[0]) for d_img in cfunc.dataset]
input_data = torch.FloatTensor(np.transpose(conv_imgs,[0,3,1,2]))
pyt_scores = model(input_data)
cls_vec = pyt_scores.detach().cpu().numpy()

In [ ]:
method_res = []

cfgs_est = []
TARGET_DIM = 250

for name, method_sol in results_Algo.items():
    if name[1] != TARGET_DIM:
        continue
    for msol in method_sol:
        clf_nn = KNeighborsRegressor(n_neighbors=1)
        clf_nn.fit(cls_vec[np.array(cfunc.balanced_split)],msol[0])
        alt_data = np.array([_ for _ in range(len(cfunc.dataset)) if _ not in cfunc.balanced_split])
        cfg_to_use = np.round(clf_nn.predict(cls_vec[alt_data])).astype(int)
        xs = np.array(msol[1])[cfg_to_use]
        res = [stereo_match(x,cfunc.dataset[i]) for x,i in zip(xs,alt_data)]
        method_res.append((name[0],name[1],np.mean(res)))
        cfgs_est.append((msol,cfg_to_use))

In [ ]:
res_f = pd.DataFrame(method_res,columns=['name','dim','res'])
res_f = res_f[res_f.dim == TARGET_DIM]
std_v = res_f.groupby(['name','dim']).std()['res']/np.sqrt(res_f.groupby(['name','dim']).count()['res'])
res_v = res_f.groupby(['name','dim']).mean().sort_values('res')
res_v['95_LB'] = res_v['res']-1.96*std_v#.reset_index()
res_v['95_UB'] = res_v['res']+1.96*std_v#.reset_index()
res_v = res_v.reset_index()
res_v 

In [ ]:
res_f.groupby(['name','dim']).count()['res']

In [ ]:
import seaborn as sns
plt.figure(figsize=(6,3))
with sns.color_palette('plasma', n_colors=len(res_v)):
    sns.boxplot(x='name',y='res',data=res_f,order=res_v.sort_values('name',0,False).name[::-1])#,flierprops={'marker':''},medianprops={'color':'k'})
plt.xlabel('')
plt.ylabel("Disparity Error")
plt.title("Stereo test set with {} function evaluations".format(TARGET_DIM))
#plt.ylim(top=1.093)
plt.tight_layout()
plt.savefig('stereo_te2st_{}_1.pdf'.format(TARGET_DIM),dpi=140,facecolor=plt.gcf().get_facecolor(), edgecolor='none',bbox_inches='tight')

In [ ]:
results_Algo

In [ ]:
min_idx = np.argsort(np.array(res_f['res']))[0]
res_f.iloc[min_idx]

In [ ]:
np.array([[np.exp(_[1]) for _ in v] for k,v in results_Algo.items() if k[0] == 'staged' and k[1] == 250]).reshape((-1,2,8))

In [ ]:


from functools import reduce

def factors(n):    
    return sorted(set(reduce(list.__add__, 
                ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0))))

for i in range(2):
    num = sum(    cfgs_est[min_idx][0][0] == i)
    if (num%2) == 1:
        num += 1

    fc = factors(num)
    if len(fc) == 1:
        fc = [fc[0],fc[0]]
    plt.figure()
    plt.suptitle('Cluster {}'.format(i+1))

    clust_idx = np.array(cfunc.balanced_split)[cfgs_est[min_idx][0][0] == i]
    for j,im_i in enumerate(clust_idx):
        plt.subplot(fc[-1],fc[-2],j+1)
        plt.imshow(cfunc.dataset[im_i][0][:,cfunc.dataset[im_i][2]:])
        plt.axis('off')
    plt.tight_layout()
    plt.subplots_adjust(wspace=-0.75, hspace=0)
    plt.savefig('stereo_c3_{}.pdf'.format(i),dpi=140,facecolor=plt.gcf().get_facecolor(), edgecolor='none',bbox_inches='tight')

In [ ]:

num = len(alt_data)
if (num%2) == 1:
    num += 1

fc = factors(num)
if len(fc) == 1:
    fc = [fc[0],fc[0]]

clust_idx = np.array(alt_data)
for j,im_i in enumerate(clust_idx):
    plt.subplot(8,3,j+1)
    plt.imshow(cfunc.dataset[im_i][0][:,cfunc.dataset[im_i][2]:])
    plt.xticks([],[])
    plt.yticks([],[])
    if cfgs_est[min_idx][1][j] == 0:
        plt.gca().patch.set_edgecolor('red')
        plt.gca().patch.set_linewidth('5')

plt.suptitle('Test Images')

plt.tight_layout()
plt.subplots_adjust(wspace=-0.75, hspace=0)
plt.savefig('stereo_c_t3_{}.pdf'.format(i),dpi=140,facecolor=plt.gcf().get_facecolor(), edgecolor='none',bbox_inches='tight')

In [ ]:
import cv2
for d_l in range(len(cfunc.dataset)):
   # plt.figure()
    if d_l in cfunc.balanced_split:
        continue
    print(d_l)
    plt.figure()
    for p_i, params in enumerate(cfgs_est[min_idx][0][1]):

        imL,imR,max_disp,mask_invalid,gt = cfunc.dataset[d_l]
        if p_i == 0:
            plt.subplot(1,3,1)
            plt.imshow(imL[:,max_disp:])
            plt.axis('off')
            plt.title("Left Image")
        p = {
            'blockSize': 5,
            'P1':8 * 3 * 3 ** 2,
            'P2':32 * 3 * 3 ** 2,
            'disp12MaxDiff': 40,
            'uniquenessRatio': 15,
            'speckleWindowSize': 0,
            'speckleRange': 2,
            'preFilterCap': 63,
        }

        p['minDisparity'] = 0
        p['numDisparities'] = max_disp
        p['mode'] = cv2.STEREO_SGBM_MODE_SGBM
        cols = ['P1', 'P2', 'blockSize', 'disp12MaxDiff', 'preFilterCap', 'speckleRange', 'speckleWindowSize', 'uniquenessRatio']

        for c,v in zip(cols,params):
            p[c] = int(round(np.exp(v)))

        left_matcher = cv2.StereoSGBM_create(**p)

        displ = left_matcher.compute(imL, imR)[:,max_disp:]
        result = (displ).astype(np.float32)/16
        max_error = 3

        err_vec = np.nan_to_num(abs(result-gt))
        err_vec[err_vec > max_error] = max_error
        err_vec[displ == -1] = max_error
        
        err = np.mean(err_vec[~mask_invalid])
        plt.subplot(1,3,p_i+2)

        plt.title('Cluster {} config \n "Vintage" ({:.3f})'.format(p_i+1,err,2))
        vis_res = result.copy()
        vis_res[(mask_invalid) | (np.isfinite(gt)==False)] = 0
        v_r = result[(result!=0) & (np.isfinite(result))]
        p5 = np.percentile(v_r,1)
        p95 = np.percentile(v_r,99)
        plt.imshow(vis_res,interpolation='nearest')
        plt.axis('off')
        
    plt.tight_layout()
    plt.savefig('stereo_out3.pdf',dpi=140,facecolor=plt.gcf().get_facecolor(), edgecolor='none',bbox_inches='tight')
    break

In [ ]:
p5,p95

In [ ]:
plt.imshow(cfunc.dataset[cfunc.balanced_split[-1]][0])

In [ ]:
plt.imshow(cfunc.dataset[cfunc.balanced_split[-1]][1])

In [ ]:
params

In [ ]:
cfunc.dataset[0]